# C1_LPF_float

In [10]:
import pandas as pd
import numpy as np
import os
from scipy.signal import butter, lfilter, freqz

In [11]:
def butter_lowpass(cutoff, fs, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter_lowpass(cutoff, fs, order=order)
    x = np.trunc(np.array(data, dtype=np.float64))
    y = lfilter(b, a, x, axis=-1, zi=None)
    return y

# Setting standard filter requirements.
order = 4
fs = 200
cutoff = 5

b, a = butter_lowpass(cutoff, fs, order)

# Creating the data for filteration
T = 200         # value taken in seconds
n = int(T * fs) # indicates total samples
t = np.linspace(0, T, n, endpoint=False)

# ADL Path

In [12]:
paths = []

for i in range(1, 20):
    root, file = os.path.split("../../Dataset/SA01/D01_SA01_R01.csv")
    if i < 10:
        file = file[:2] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        paths.append(tmp)
        if i > 4:
            for j in range(2, 6):
                file_new = file[:-5] + str(j) + file[-4:]
                tmp = os.path.join(root, file_new)
                paths.append(tmp)
    if i >= 10:
        file = file[:1] + str(i) + file[3:]
        for j in range(1, 6):
            file = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file)
            paths.append(tmp)

paths[4:14] = [ '../../Dataset/SA01/D05_SA01_R01_1.csv',
                '../../Dataset/SA01/D05_SA01_R01_2.csv',
                '../../Dataset/SA01/D05_SA01_R02_1.csv',
                '../../Dataset/SA01/D05_SA01_R02_2.csv',
                '../../Dataset/SA01/D05_SA01_R03_1.csv',
                '../../Dataset/SA01/D05_SA01_R03_2.csv',
                '../../Dataset/SA01/D05_SA01_R04_1.csv',
                '../../Dataset/SA01/D05_SA01_R04_2.csv',
                '../../Dataset/SA01/D05_SA01_R05_1.csv',
                '../../Dataset/SA01/D05_SA01_R05_2.csv',
                '../../Dataset/SA01/D06_SA01_R01_1.csv',
                '../../Dataset/SA01/D06_SA01_R01_2.csv',
                '../../Dataset/SA01/D06_SA01_R02_1.csv',
                '../../Dataset/SA01/D06_SA01_R02_2.csv',
                '../../Dataset/SA01/D06_SA01_R03_1.csv',
                '../../Dataset/SA01/D06_SA01_R03_2.csv',
                '../../Dataset/SA01/D06_SA01_R04_1.csv',
                '../../Dataset/SA01/D06_SA01_R04_2.csv',
                '../../Dataset/SA01/D06_SA01_R05_1.csv',
                '../../Dataset/SA01/D06_SA01_R05_2.csv']

paths[-15:-10] =['../../Dataset/SA01/D17_SA01_R01_1.csv',
                 '../../Dataset/SA01/D17_SA01_R01_2.csv',
                 '../../Dataset/SA01/D17_SA01_R02_1.csv',
                 '../../Dataset/SA01/D17_SA01_R02_2.csv',
                 '../../Dataset/SA01/D17_SA01_R03_1.csv',
                 '../../Dataset/SA01/D17_SA01_R03_2.csv',
                 '../../Dataset/SA01/D17_SA01_R04_1.csv',
                 '../../Dataset/SA01/D17_SA01_R04_2.csv',
                 '../../Dataset/SA01/D17_SA01_R05_1.csv',
                 '../../Dataset/SA01/D17_SA01_R05_2.csv']


subject_list = ['SA01', 'SA02', 'SA03', 'SA04', 'SA05',
                'SA06', 'SA07', 'SA08', 'SA09', 'SA10',
                'SA11', 'SA12', 'SA13', 'SA14', 'SA15',
                'SA16', 'SA17', 'SA18', 'SA19', 'SA20',
                'SA21', 'SA22', 'SA23',
                'SE01', 'SE02', 'SE03', 'SE04', 'SE05',
                'SE06', 'SE07', 'SE08', 'SE09', 'SE10',
                'SE11', 'SE12', 'SE13', 'SE14', 'SE15']


total_adl_path = []

for path in paths:
    for subject_name in subject_list:
        path_rename = path.replace('SA01', subject_name)
        total_adl_path.append(path_rename)

len(total_adl_path)

3572

# ADL data 전처리

In [4]:
adl_list = []

for path in total_adl_path:
    try:
        data = pd.read_csv(path, header=None)
        data = data.iloc[:, :3]
        data = np.abs(data)
                    
        max_value1 = data[0][200:].max()
        max_value2 = data[1][200:].max()
        max_value3 = data[2][200:].max()
        
        max_idx1 = data[ data[0] == max_value1 ].index[0]
        max_idx2 = data[ data[1] == max_value2 ].index[0]
        max_idx3 = data[ data[2] == max_value3 ].index[0]
        
        min_idx = min(max_idx1, max_idx2, max_idx3)
        sliced_df = data[min_idx - 200 : min_idx - 40]
        
        col_1_lpf = butter_lowpass_filter(sliced_df[0].to_numpy(), cutoff, fs, order).astype('float64')
        col_2_lpf = butter_lowpass_filter(sliced_df[1].to_numpy(), cutoff, fs, order).astype('float64')
        col_3_lpf = butter_lowpass_filter(sliced_df[2].to_numpy(), cutoff, fs, order).astype('float64')

        col_1 = col_1_lpf*col_1_lpf
        col_2 = col_2_lpf*col_2_lpf
        col_3 = col_3_lpf*col_3_lpf
        
        C_nine = np.sqrt(col_1 + col_2 + col_3).tolist()
        adl_list.append(C_nine)
        # adl_first = pd.concat([list_C_nine, adl_first], axis=0, ignore_index=True)
         
    except:
        pass
    

adl_df = pd.DataFrame(adl_list)
print(adl_df.shape)
row, col = adl_df.shape


target_col = row * [0]
adl_df['target'] = target_col

print(adl_df.shape)
adl_df

(3209, 160)
(3209, 161)


,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,target
0,0.011319,0.096369,0.410542,1.195297,2.729051,5.278112,9.063773,14.243724,20.904497,29.060575,...,233.985140,230.841883,227.811549,224.938888,222.267447,219.836665,217.679713,215.822139,214.281083,0
1,0.008661,0.074225,0.318734,0.936395,2.158682,4.217257,7.317843,11.623518,17.244860,24.235002,...,185.512330,185.028192,184.976320,185.332663,186.065617,187.136945,188.502741,190.115054,191.924383,0
2,0.010328,0.088726,0.381948,1.125228,2.602263,5.101379,8.882013,14.150984,21.047922,29.637897,...,260.482562,258.556009,256.394214,254.025992,251.488867,248.827409,246.092027,243.338405,240.626526,0
3,0.008268,0.071051,0.305998,0.901905,2.086652,4.092138,7.128366,11.366145,16.926643,23.876111,...,253.463132,252.924844,252.256294,251.480991,250.622928,249.705943,248.752983,247.784891,246.819315,0
4,0.006762,0.057793,0.247591,0.726108,1.672177,3.265694,5.667655,9.006618,13.370635,18.804320,...,226.108608,226.239583,226.091009,225.661758,224.966738,224.036220,222.913739,221.653135,220.315759,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3204,0.008093,0.069585,0.299912,0.884922,2.050419,4.028986,7.035315,11.248836,16.802210,23.774888,...,204.998980,199.653769,193.675526,187.121198,180.039196,172.474644,164.471202,156.070976,147.315105,0
3205,0.008109,0.069617,0.299585,0.882675,2.042589,4.009034,6.993190,11.170828,16.671767,23.574077,...,263.174856,262.883269,262.552820,262.167243,261.703520,261.132045,260.417195,259.517666,258.386411,0
3206,0.007982,0.068594,0.295342,0.870252,2.013350,3.950007,6.886981,10.995703,16.401656,23.178988,...,69.265076,61.999309,58.408597,56.861950,56.636126,57.994566,61.730242,68.469177,78.199512,0
3207,0.007877,0.067664,0.291397,0.859339,1.990787,3.912623,6.835834,10.938221,16.352427,23.159231,...,83.141189,76.917148,70.289400,63.509599,56.895483,50.833091,45.770135,42.169289,40.384283,0


# ADL data 저장

In [5]:
final = pd.DataFrame(adl_df)
final.to_csv('../result/a_c1_LPF_float.csv', index=False)

# Fall path

In [6]:
fall_list = []

for i in range(1, 16):
    root, file = os.path.split("../../Dataset/SA01/F01_SA01_R01.csv")
    if i < 10:
        file = file[:2] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        for j in range(1, 6):
            file_new = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file_new)
            fall_list.append(tmp)
    if i >= 10:
        file = file[:1] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        for j in range(1, 6):
            file_new = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file_new)
            fall_list.append(tmp)
            
print(fall_list)

subject_list = ['SA01', 'SA02', 'SA03', 'SA04', 'SA05',
                'SA06', 'SA07', 'SA08', 'SA09', 'SA10',
                'SA11', 'SA12', 'SA13', 'SA14', 'SA15',
                'SA16', 'SA17', 'SA18', 'SA19', 'SA20',
                'SA21', 'SA22', 'SA23', 'SE06']

total_fall_path = []

for path in fall_list:
    for subject_name in subject_list:
        path_rename = path.replace('SA01', subject_name)
        total_fall_path.append(path_rename)

total_fall_path

['../../Dataset/SA01/F01_SA01_R01.csv', '../../Dataset/SA01/F01_SA01_R02.csv', '../../Dataset/SA01/F01_SA01_R03.csv', '../../Dataset/SA01/F01_SA01_R04.csv', '../../Dataset/SA01/F01_SA01_R05.csv', '../../Dataset/SA01/F02_SA01_R01.csv', '../../Dataset/SA01/F02_SA01_R02.csv', '../../Dataset/SA01/F02_SA01_R03.csv', '../../Dataset/SA01/F02_SA01_R04.csv', '../../Dataset/SA01/F02_SA01_R05.csv', '../../Dataset/SA01/F03_SA01_R01.csv', '../../Dataset/SA01/F03_SA01_R02.csv', '../../Dataset/SA01/F03_SA01_R03.csv', '../../Dataset/SA01/F03_SA01_R04.csv', '../../Dataset/SA01/F03_SA01_R05.csv', '../../Dataset/SA01/F04_SA01_R01.csv', '../../Dataset/SA01/F04_SA01_R02.csv', '../../Dataset/SA01/F04_SA01_R03.csv', '../../Dataset/SA01/F04_SA01_R04.csv', '../../Dataset/SA01/F04_SA01_R05.csv', '../../Dataset/SA01/F05_SA01_R01.csv', '../../Dataset/SA01/F05_SA01_R02.csv', '../../Dataset/SA01/F05_SA01_R03.csv', '../../Dataset/SA01/F05_SA01_R04.csv', '../../Dataset/SA01/F05_SA01_R05.csv', '../../Dataset/SA01/F06_

['../../Dataset/SA01/F01_SA01_R01.csv',
 '../../Dataset/SA02/F01_SA02_R01.csv',
 '../../Dataset/SA03/F01_SA03_R01.csv',
 '../../Dataset/SA04/F01_SA04_R01.csv',
 '../../Dataset/SA05/F01_SA05_R01.csv',
 '../../Dataset/SA06/F01_SA06_R01.csv',
 '../../Dataset/SA07/F01_SA07_R01.csv',
 '../../Dataset/SA08/F01_SA08_R01.csv',
 '../../Dataset/SA09/F01_SA09_R01.csv',
 '../../Dataset/SA10/F01_SA10_R01.csv',
 '../../Dataset/SA11/F01_SA11_R01.csv',
 '../../Dataset/SA12/F01_SA12_R01.csv',
 '../../Dataset/SA13/F01_SA13_R01.csv',
 '../../Dataset/SA14/F01_SA14_R01.csv',
 '../../Dataset/SA15/F01_SA15_R01.csv',
 '../../Dataset/SA16/F01_SA16_R01.csv',
 '../../Dataset/SA17/F01_SA17_R01.csv',
 '../../Dataset/SA18/F01_SA18_R01.csv',
 '../../Dataset/SA19/F01_SA19_R01.csv',
 '../../Dataset/SA20/F01_SA20_R01.csv',
 '../../Dataset/SA21/F01_SA21_R01.csv',
 '../../Dataset/SA22/F01_SA22_R01.csv',
 '../../Dataset/SA23/F01_SA23_R01.csv',
 '../../Dataset/SE06/F01_SE06_R01.csv',
 '../../Dataset/SA01/F01_SA01_R02.csv',


# Fall 데이터 전처리

In [7]:
fall_list = []

for path in total_fall_path:
    try:
        data = pd.read_csv(path, header=None)
        data = data.iloc[:, :3]
        data = np.abs(data)
                    
        max_value1 = data[0][1000:].max()
        max_value2 = data[1][1000:].max()
        max_value3 = data[2][1000:].max()
        
        max_idx1 = data[ data[0] == max_value1 ].index[0]
        max_idx2 = data[ data[1] == max_value2 ].index[0]
        max_idx3 = data[ data[2] == max_value3 ].index[0]
        
        min_idx = min(max_idx1, max_idx2, max_idx3)
        sliced_df = data[min_idx - 200 : min_idx - 40]
        
        col_1_lpf = butter_lowpass_filter(sliced_df[0].to_numpy(), cutoff, fs, order).astype('float64')
        col_2_lpf = butter_lowpass_filter(sliced_df[1].to_numpy(), cutoff, fs, order).astype('float64')
        col_3_lpf = butter_lowpass_filter(sliced_df[2].to_numpy(), cutoff, fs, order).astype('float64')

        col_1 = col_1_lpf*col_1_lpf
        col_2 = col_2_lpf*col_2_lpf
        col_3 = col_3_lpf*col_3_lpf
        
        C_nine = np.sqrt(col_1 + col_2 + col_3).tolist()
        fall_list.append(C_nine)
        # adl_first = pd.concat([list_C_nine, adl_first], axis=0, ignore_index=True)
         
    except:
        pass
    

fall_df = pd.DataFrame(fall_list)
print(fall_df.shape)
row, col = fall_df.shape


target_col = row * [1]
fall_df['target'] = target_col

print(fall_df.shape)
fall_df

(1799, 160)
(1799, 161)


,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,target
0,0.005912,0.050826,0.219054,0.646589,1.499943,2.954185,5.177664,8.321280,12.510633,17.840207,...,132.585645,127.706643,126.577157,129.022931,134.522440,142.340464,151.686261,161.822131,172.111492,1
1,0.006833,0.058867,0.254416,0.753474,1.753956,3.464683,6.083882,9.783274,14.697452,20.916996,...,185.859147,181.099017,176.511588,172.175141,168.167667,164.564073,161.432881,158.832304,156.806181,1
2,0.007933,0.068246,0.294420,0.869760,2.017993,3.970937,6.944217,11.119681,16.633633,23.570400,...,179.345160,185.215660,190.937068,196.495174,201.891394,207.135387,212.237981,217.204197,222.027937,1
3,0.006882,0.059067,0.254148,0.748817,1.733143,3.402496,5.936327,9.483482,14.153160,20.010053,...,149.959666,142.452140,135.391623,129.005041,123.482255,118.975434,115.619279,113.570164,113.044238,1
4,0.008461,0.072461,0.310616,0.910481,2.094874,4.088865,7.098155,11.295593,16.814110,23.744770,...,231.103803,238.921337,246.927252,254.865925,262.452759,269.391647,275.396752,280.214123,283.641140,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1794,0.007914,0.067949,0.292565,0.862749,1.998726,3.927806,6.859974,10.970663,16.389623,23.194678,...,212.920788,211.451489,210.070003,208.775430,207.561143,206.416392,205.327687,204.279770,203.256659,1
1795,0.007941,0.068162,0.293167,0.863076,1.995579,3.914134,6.824581,10.898758,16.264046,22.997585,...,254.424441,254.697961,254.323829,253.354995,251.882288,250.024441,247.916694,245.701500,243.522217,1
1796,0.008075,0.069276,0.297893,0.876948,2.027520,3.976012,6.930131,11.062138,16.498147,23.312511,...,191.629555,187.454219,183.066921,178.568219,174.087781,169.783640,165.837262,162.443282,159.794288,1
1797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


# Fall 데이터 저장

In [8]:
final = pd.DataFrame(fall_df)
final.to_csv('../result/f_c1_LPF_float.csv', index=False)

# All 데이터 저장

In [9]:
adl = pd.read_csv('../result/a_c1_LPF_float.csv')
fall = pd.read_csv('../result/f_c1_LPF_float.csv')

fall_data = pd.DataFrame(fall)
print(fall_data.shape)
adl_data = pd.DataFrame(adl)
print(adl_data.shape)

file_list = [fall_data, adl_data]
result = pd.concat(file_list)
result.to_csv('../result/c1_LPF_float.csv', index=False)
print(result.shape)
result

(1799, 161)
(3209, 161)
(5008, 161)


,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,target
0,0.005912,0.050826,0.219054,0.646589,1.499943,2.954185,5.177664,8.321280,12.510633,17.840207,...,132.585645,127.706643,126.577157,129.022931,134.522440,142.340464,151.686261,161.822131,172.111492,1
1,0.006833,0.058867,0.254416,0.753474,1.753956,3.464683,6.083882,9.783274,14.697452,20.916996,...,185.859147,181.099017,176.511588,172.175141,168.167667,164.564073,161.432881,158.832304,156.806181,1
2,0.007933,0.068246,0.294420,0.869760,2.017993,3.970937,6.944217,11.119681,16.633633,23.570400,...,179.345160,185.215660,190.937068,196.495174,201.891394,207.135387,212.237981,217.204197,222.027937,1
3,0.006882,0.059067,0.254148,0.748817,1.733143,3.402496,5.936327,9.483482,14.153160,20.010053,...,149.959666,142.452140,135.391623,129.005041,123.482255,118.975434,115.619279,113.570164,113.044238,1
4,0.008461,0.072461,0.310616,0.910481,2.094874,4.088865,7.098155,11.295593,16.814110,23.744770,...,231.103803,238.921337,246.927252,254.865925,262.452759,269.391647,275.396752,280.214123,283.641140,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3204,0.008093,0.069585,0.299912,0.884922,2.050419,4.028986,7.035315,11.248836,16.802210,23.774888,...,204.998980,199.653769,193.675526,187.121198,180.039196,172.474644,164.471202,156.070976,147.315105,0
3205,0.008109,0.069617,0.299585,0.882675,2.042589,4.009034,6.993190,11.170828,16.671767,23.574077,...,263.174856,262.883269,262.552820,262.167243,261.703520,261.132045,260.417195,259.517666,258.386411,0
3206,0.007982,0.068594,0.295342,0.870252,2.013350,3.950007,6.886981,10.995703,16.401656,23.178988,...,69.265076,61.999309,58.408597,56.861950,56.636126,57.994566,61.730242,68.469177,78.199512,0
3207,0.007877,0.067664,0.291397,0.859339,1.990787,3.912623,6.835834,10.938221,16.352427,23.159231,...,83.141189,76.917148,70.289400,63.509599,56.895483,50.833091,45.770135,42.169289,40.384283,0
